__Programming Assignment #2__ <br>
_ORI 391Q: Linear Programming_  <br>

Luis Botello <br>
LB34352 <br>
9/27/22

In [38]:
#Import Libraries
import scipy.sparse as sparse
import numpy as np
from scipy.linalg import lu_factor, lu_solve
from scipy.linalg import cho_factor, cho_solve

In [3]:
#Random Matrix Constructor Class
class matrix_gen:
    def __init__(self, rows, columns, density, lower_bound, upper_bound):
        self.rows = rows
        self.columns = columns
        self.density = density
        self.upper_bound = upper_bound
        self.lower_bound = lower_bound

    def norm_matrix(self):  #generate matrix using user-defined dimensions and density
        np.random.seed(1)  #for testing
        return sparse.random(self.rows, self.columns, density=self.density).todense()

    def scaled_matrix(self):  #scaled norm_matrix using user-defined bounds
        test = self.norm_matrix()
        return np.where(test>0,
                        (test*(self.upper_bound - self.lower_bound))+self.lower_bound,
                        test)

In [4]:
#Generate Random Matrix, B with dimensions: 10x10 and density = 0.8
B = matrix_gen(10,10,.8,-10,30).scaled_matrix()

In [5]:
#Check for all-zero rows and columns & for singularity.
#If requirements are not met, generate new matrix of same dimension and density until requirements are met
counter = 1
while (~np.any(B>0, axis = 0)).any() == True or (~np.any(B>0, axis = 1)).any() == True or np.linalg.det(B) == 0:
    counter += 1
    print("Matrix is singular or an all-Zero row or column was detected, trial #:", counter)
    B = matrix_gen(10,10,.8,-10,30).scaled_matrix()

In [23]:
print('Matrix B:')
print('Determinant=', np.linalg.det(B))
print(np.round(B,decimals=2))

Matrix B:
Determinant= -63198234235463.46
[[22.52  4.26  0.   -0.73 13.15  0.    4.71  1.13 -8.19  1.59]
 [ 0.    0.   -5.71 15.91  8.9  -4.95  7.08  0.   -8.81 -2.32]
 [20.11  0.   14.52 17.61 -0.66 15.51  9.92  0.   25.69 16.09]
 [-5.91  4.48  8.83 21.7  10.33  0.76 20.07 25.25 -7.2  25.33]
 [10.75 26.66 -8.03 20.58  4.16  2.05  0.    1.55 -4.4  -7.44]
 [ 0.   12.05  0.   21.48 19.71 -1.66 11.54  0.   27.49 20.91]
 [ 0.    0.    8.57  7.92 27.67 12.84 28.71  0.    8.7   7.14]
 [ 7.11 11.63  4.21  0.   -6.34 18.7  20.52 23.27 10.39 -9.53]
 [ 6.72  0.    0.    5.19 -9.64 -3.11 -3.87 20.53 -3.88 -0.79]
 [ 0.   -7.05  0.    4.45  5.85 18.36  5.1   0.   12.8  20.86]]


In [24]:
#Generate Random Matrix, b with dimensions: 10x1 and density = 0.8
b = matrix_gen(10,1,.8,0,50).scaled_matrix()
print('Matrix, b:')
print(np.round(b,decimals=3))

Matrix, b:
[[15.664]
 [22.173]
 [19.396]
 [26.227]
 [42.316]
 [ 0.   ]
 [46.777]
 [11.479]
 [ 0.   ]
 [33.487]]


##### __Method 1: LU Factorization__

In [47]:
#Method 1: LU Decomposition
lu, piv = lu_factor(B)
x = lu_solve((lu,piv), b)

print('Solution via LU Decomposition')
print(np.round(x, decimals=3))

Solution via LU Decomposition
[[-0.363]
 [-0.286]
 [ 0.935]
 [ 1.641]
 [ 2.183]
 [ 2.959]
 [-1.995]
 [ 0.582]
 [-0.96 ]
 [-0.98 ]]


##### __Method 2: Scipy Routine that Solves Bx = b Directly__

In [66]:
x = scipy.linalg.solve(B, b)

print('Solution via Scipy .solve method')
print(np.round(x, decimals=3))

Solution via Scipy .solve method
[[-0.363]
 [-0.286]
 [ 0.935]
 [ 1.641]
 [ 2.183]
 [ 2.959]
 [-1.995]
 [ 0.582]
 [-0.96 ]
 [-0.98 ]]


##### __Method 2: Inverse of Matrix B, and Dot Product__

In [67]:
#Inverse matrix B, using numpy .inv() function.
inv_B = np.linalg.inv(B)

In [49]:
#Solve using dot product.
x  = np.dot(inv_B,b)

print('Solution via Inverse of B')
print(np.round(x,decimals=3))

Solution via Inverse of B
[[-0.363]
 [-0.286]
 [ 0.935]
 [ 1.641]
 [ 2.183]
 [ 2.959]
 [-1.995]
 [ 0.582]
 [-0.96 ]
 [-0.98 ]]


##### __Method 4: Cholesky Factors__

In [76]:
D = 0.5*(B + np.matrix.transpose(B))

In [77]:
#Function to check if Matrix is Positive Definitive
def is_pos_def(A):
    try:
        scipy.linalg.cholesky(A)
        return True
    except scipy.linalg.LinAlgError:
        return False

In [78]:
#Check if D is positive definitive 
is_pos_def(D)

False

In [79]:
for i in range(len(D)):
    D[i,i] = D[i,i] + 100

In [80]:
#Solves for x using Cholesky factors
c, low = cho_factor(D)
x = cho_solve((c, low), b)

print('Solution via Cholesky Factors')
print(np.round(x,decimals=3))

Solution via Cholesky Factors
[[ 0.072]
 [ 0.158]
 [ 0.136]
 [ 0.084]
 [ 0.346]
 [-0.164]
 [ 0.298]
 [ 0.078]
 [-0.001]
 [ 0.28 ]]
